# Building an AI Book Research Assistant

This notebook provides step-by-step instructions for replicating what Andrew did in the second demo. Please follow along, and feel free to play with different variations too! At the end, there are optional instructions for building a more advanced book research assistant than Andrew showed. 

*Useful Resources*:

- **AISuite GitHub Repo**: [https://github.com/andrewyng/aisuite](https://github.com/andrewyng/aisuite)
- **Open Library API**:
  - [Open Library](https://openlibrary.org/developers/api) offers free and public Web APIs for accessing book and author catalog data (no API key is required)
  - See `openlibrary_api_docs.md` in this lesson's directory for more information

## Step 1: Use Open Library API's Documentation

- Open Jupyter chat by clicking on the chat bubble icon on the left sidebar of Jupyter Lab
- In the Chat interface, create a new chat by clicking `+Chat` (choose any name for the file)
- In the chat window, attach the Markdown file `openlibrary_api_docs.md` using the `@` symbol:
  - Type `@`, then select `file` from the autocomplete menu
  - You'll see a list of available files in the lesson's directory, choose `openlibrary_api_docs.md` 
- Use a prompt like this:
   > Write a code cell that uses Open Library's search endpoint to look up books based on a user's query and returns a response listing identified books, with these fields for each book: key, title, author_name, first_publish_year, isbn, cover_id, edition_count, language, subject. Use a variable called limit, set to 5, to specify the number of returned results. Print the results. Use the Open Library API documentation in the attached file.
- Transfer the generated code to the cell below and run it

In [1]:
import requests

# User query and parameters
query = input("Enter a search query: ").strip()
limit = 5  # number of results to return

# API endpoint and parameters
url = "https://openlibrary.org/search.json"
params = {
    "q": query,
    "limit": limit,
    "fields": "key,title,author_name,first_publish_year,isbn,cover_i,edition_count,language,subject"
}

# Send GET request to Open Library Search API
response = requests.get(url, params=params)
response.raise_for_status()
data = response.json()

# Print formatted book results
for i, book in enumerate(data.get("docs", []), start=1):
    print(f"\nBook {i}:")
    print(f"  Key: {book.get('key')}")
    print(f"  Title: {book.get('title')}")
    print(f"  Author(s): {', '.join(book.get('author_name', ['Unknown']))}")
    print(f"  First Publish Year: {book.get('first_publish_year', 'N/A')}")
    print(f"  ISBN(s): {', '.join(book.get('isbn', [])[:3]) if book.get('isbn') else 'N/A'}")
    print(f"  Cover ID: {book.get('cover_i', 'N/A')}")
    print(f"  Edition Count: {book.get('edition_count', 'N/A')}")
    print(f"  Language(s): {', '.join(book.get('language', [])) if book.get('language') else 'N/A'}")
    print(f"  Subject(s): {', '.join(book.get('subject', [])[:5]) if book.get('subject') else 'N/A'}")

Enter a search query:  tennis



Book 1:
  Key: /works/OL81632W
  Title: 'Salem’s Lot
  Author(s): Stephen King
  First Publish Year: 1975
  ISBN(s): 9780307743671, 9780345806796, 0385516487
  Cover ID: 14654118
  Edition Count: 105
  Language(s): pol, spa, ita, por, rus, eng, ger, tur, fre
  Subject(s): Catholic Church, arson, Great Depression, hitmen, Horror fiction

Book 2:
  Key: /works/OL551668W
  Title: Cyrano de Bergerac
  Author(s): Edmond Rostand
  First Publish Year: 1821
  ISBN(s): 0451525485, 9781557832306, 9780141961071
  Cover ID: 8236320
  Edition Count: 308
  Language(s): heb, sco, ara, spa, cat, und, chi, slo, rus, eng, ger, tgl, fre
  Subject(s): Drama, History, French Authors, Cyrano de Bergerac, Savinien, 1619-1655, French drama

Book 3:
  Key: /works/OL769120W
  Title: Book of common prayer
  Author(s): Church of England, J. A. Maurault
  First Publish Year: 1537
  ISBN(s): 9780665146497, 9780665011160, 9780665146503
  Cover ID: 5720552
  Edition Count: 545
  Language(s): mar, wel, eng, spa, tur,

## Step 2: Wrap the API Call in a Tool Function

- In the chat interface, use a prompt like this:
  > Wrap the code above in a function `search_books`, with good docstring.
  > 
  > - Input arguments: `query` and `limit` (with default value of 10)
  > - Output: JSON string with:
  >   - `num_found`: total results
  >   - `books`: list of dicts with title, authors, first_publish_year, key, edition_count

In [2]:
import requests
import json

def search_books(query: str, limit: int = 10) -> str:
    """
    Search for books in the Open Library database using the Open Library Search API.
    
    Args:
        query (str): The user's search query (e.g., title, author, subject, etc.).
        limit (int, optional): The number of results to return. Default is 10.
    
    Returns:
        str: A JSON-formatted string containing:
            - num_found: The total number of results found by the API.
            - books: A list of dictionaries, each containing:
                - title (str): The title of the book.
                - authors (list[str]): List of author names.
                - first_publish_year (int or None): The first publish year.
                - key (str): The unique Open Library work key.
                - edition_count (int or None): The number of editions available.
    """
    url = "https://openlibrary.org/search.json"
    params = {
        "q": query,
        "limit": limit,
        "fields": "key,title,author_name,first_publish_year,edition_count"
    }

    response = requests.get(url, params=params)
    response.raise_for_status()
    data = response.json()

    # Extract required information
    books = []
    for doc in data.get("docs", []):
        books.append({
            "title": doc.get("title"),
            "authors": doc.get("author_name", []),
            "first_publish_year": doc.get("first_publish_year"),
            "key": doc.get("key"),
            "edition_count": doc.get("edition_count")
        })

    result = {
        "num_found": data.get("num_found", 0),
        "books": books
    }

    return json.dumps(result, indent=2)



In [3]:
# Example usage:
print(search_books("Python programming", limit=5))

{
  "num_found": 2485,
  "books": [
    {
      "title": "Core Python Programming",
      "authors": [
        "R. Nageswara Rao"
      ],
      "first_publish_year": 2016,
      "key": "/works/OL30906747W",
      "edition_count": 5
    },
    {
      "title": "Core Python programming",
      "authors": [
        "Wesley Chun"
      ],
      "first_publish_year": 2006,
      "key": "/works/OL12409294W",
      "edition_count": 2
    },
    {
      "title": "Black Hat Python",
      "authors": [
        "Justin Seitz",
        "Tim Arnold"
      ],
      "first_publish_year": 2014,
      "key": "/works/OL19547345W",
      "edition_count": 3
    },
    {
      "title": "Python Programming",
      "authors": [
        "Reema Thareja"
      ],
      "first_publish_year": 2019,
      "key": "/works/OL26755050W",
      "edition_count": 2
    },
    {
      "title": "Python",
      "authors": [
        "Joshua Welsh"
      ],
      "first_publish_year": 2017,
      "key": "/works/OL20212390W",

- Test the function by uncommenting the code below:

In [4]:
results = search_books("science fiction")

## Step 3: Build a Chat Handler with Automatic Function Calling

- Drag and drop the raw cell below into the chat window and use a prompt like this:
  > Use this aisuite example to create a simple chat handler function `chat_with_tools`:
  > - input: `user_message`
  > - output: `response.choices[0].message.content`

In [7]:
import aisuite as ai

# Initialize the AISuite client
client = ai.Client()

def chat_with_tools(user_message: str) -> str:
    """
    A simple chat handler that processes user messages and can automatically
    call registered tools such as search_books.
    
    Args:
        user_message (str): A natural language message from the user.
    
    Returns:
        str: The assistant's generated response text.
    """
    messages = [
        {
            "role": "user",
            "content": user_message
        }
    ]
    
    # Automatic tool execution with a few back-and-forth turns
    response = client.chat.completions.create(
        model="openai:gpt-4.1-mini",
        messages=messages,
        tools=[search_books],  # tools available for the chat
        max_turns=3
    )
    
    return response.choices[0].message.content



In [8]:
# Example usage:
reply = chat_with_tools("Find books about artificial intelligence")
print(reply)

Here are some books about artificial intelligence:

1. "Artificial Intelligence" by Stuart J. Russell and Peter Norvig (First published in 1994)
2. "Artificial Intelligence" by Patrick Henry Winston (First published in 1977)
3. "Artificial Intelligence" by Elaine A. Rich (First published in 1983)
4. "Artificial Intelligence and Intelligent Systems" by N. P. Padhy (First published in 2005)
5. "Artificial Intelligence and Soft Computing" by Leszek Rutkowski (First published in 2012)
6. "Artificial Intelligence" by Melanie Mitchell (First published in 2019)
7. "Understanding Artificial Intelligence" by Henry C. Mishkoff (First published in 1985)
8. "Artificial Intelligence Book" by Long Hileman (First published in 2021)
9. "Artificial Intelligence" by K. Warwick (First published in 2012)
10. "Human Compatible" by Stuart J. Russell (First published in 2019)

If you want more details about any of these books or want to find more, please let me know!


## Step 4: Test the Chat Handler

- Run the following code cell to test `chat_with_tools`
- Feel free to search for different types of books
- Optional: If your generated function `search_books` does not contain printing statements, you can update the function as Andrew showed in the video:
  - prompt used: modify the search_books function to add a printing statement showing when it is called with basic status messages.

In [9]:
print("="*60)
print("EXAMPLE: Science Function book search")
print("="*60)

user_query = "Search for books about science fiction"
print(f"\n👤 User: {user_query}")

response = chat_with_tools(user_query)

print(f"\n🤖 Assistant: {response}")

EXAMPLE: Science Function book search

👤 User: Search for books about science fiction

🤖 Assistant: Here are some books about science fiction:

1. The Science Fiction Hall of Fame -- Volume One by Robert Silverberg, Robert A. Heinlein, Arthur C. Clarke, Isaac Asimov, Ray Bradbury (First published in 1970)
2. Foundation by Isaac Asimov (First published in 1951)
3. Stranger in a Strange Land by Robert A. Heinlein (First published in 1961)
4. The Last Man by Mary Shelley (First published in 1826)
5. The Time Machine by H. G. Wells (First published in 1895)
6. Dune by Frank Herbert (First published in 1965)
7. The Invisible Man by H. G. Wells
8. Мы by Евгений Иванович Замятин and others (First published in 1924)
9. Wild Seed by Octavia E. Butler (First published in 1980)
10. Great Sf Heinlein Bxs by Robert A. Heinlein (First published in 1980)

If you want more information about any specific book or another list, please let me know!


## Step 5: Look up Details on Books (optional) 

You can repeat steps 1-3 to define a second tool for the book research assistant, so it can handle more complex book queries.
For example, you can use the `works` endpoint to retrieve more detailed information about a particular book given its work id. For more details, check `openlibrary_api_docs.md`.

Here are steps with suggested prompts:

1. In the chat interface, attach the file containing the API documentation of Open Library (`openlibrary_api_docs.md`) and use a prompt like this:
   > Write a code cell that uses Open Library's works endpoint to retrieve information about a book, based on the work's key that has this format: "/works/{work_id}". The result should include: title, description, subjects (limit to 10), first_sentence, covers (limit to 3). Print the returned results. Use the documentation for Open Library's API provided above.
2. In the chat interface, create the function tool using a prompt like this:
   > Wrap the code above in a function `get_book_details`:
   > - Function argument: `work_key` in this format: `/works/{work_id}`
   > - Output: JSON string with: title, description, subjects (limit to 10), first_sentence, covers (limit to 3)
   >  - Write good docstring
3. Update the `chat_with_tools` function to include the second tool as follows:
   
   ``` python
   response = client.chat.completions.create(
    model="openai:gpt-4.1-mini",
    messages=messages,
    tools=[search_books, get_book_details],
    max_turns=3  # Maximum number of back-and-forth tool calls
   )
   ```
   
4. Try this query:
   ```python
   user_query = "Search for books with 'Linear Algebra' in the title and tell me about the one with the most editions"
   ```

In [10]:
import requests

def get_work_details(work_key: str) -> dict:
    """
    Retrieve detailed information about a specific work from the Open Library Works API.

    Args:
        work_key (str): The work key in the format "/works/{work_id}", e.g., "/works/OL14933374W".

    Returns:
        dict: A dictionary containing:
            - title (str): The work's title.
            - description (str): A textual description of the work, if available.
            - subjects (list[str]): Up to 10 subject tags.
            - first_sentence (str): The first sentence of the work, if available.
            - covers (list[int]): Up to 3 cover image IDs.
    """
    # Construct full endpoint
    base_url = "https://openlibrary.org"
    url = f"{base_url}{work_key}.json"
    
    response = requests.get(url)
    response.raise_for_status()
    data = response.json()
    
    # Extract required fields safely
    result = {
        "title": data.get("title"),
        "description": (
            data["description"]["value"]
            if isinstance(data.get("description"), dict)
            else data.get("description")
        ),
        "subjects": data.get("subjects", [])[:10],
        "first_sentence": (
            data["first_sentence"]["value"]
            if isinstance(data.get("first_sentence"), dict)
            else data.get("first_sentence")
        ),
        "covers": data.get("covers", [])[:3]
    }
    
    print(result)
    return result



In [11]:
# Example usage:
get_work_details("/works/OL14933374W")

{'title': 'An English forest', 'description': None, 'subjects': [], 'first_sentence': None, 'covers': [5688862, 7189461]}


{'title': 'An English forest',
 'description': None,
 'subjects': [],
 'first_sentence': None,
 'covers': [5688862, 7189461]}

In [12]:
import requests
import json

def get_book_details(work_key: str) -> str:
    """
    Retrieve detailed book information from the Open Library Works API.

    This function queries the Open Library's `/works/{work_id}.json` endpoint and extracts
    key details about the book. It safely handles optional fields and ensures lists are truncated
    to the specified limits.

    Args:
        work_key (str): The work key of the book in the format "/works/{work_id}".
                        Example: "/works/OL14933374W"

    Returns:
        str: A JSON-formatted string containing:
            - title (str): The book's title
            - description (str): Textual description of the book (if available)
            - subjects (list[str]): Up to 10 subject tags
            - first_sentence (str): The first sentence of the book (if available)
            - covers (list[int]): Up to 3 cover IDs
    """
    base_url = "https://openlibrary.org"
    url = f"{base_url}{work_key}.json"
    
    response = requests.get(url)
    response.raise_for_status()
    data = response.json()

    # Safely extract fields
    result = {
        "title": data.get("title"),
        "description": (
            data["description"]["value"]
            if isinstance(data.get("description"), dict)
            else data.get("description")
        ),
        "subjects": data.get("subjects", [])[:10],
        "first_sentence": (
            data["first_sentence"]["value"]
            if isinstance(data.get("first_sentence"), dict)
            else data.get("first_sentence")
        ),
        "covers": data.get("covers", [])[:3],
    }

    return json.dumps(result, indent=2)


In [13]:

# Example usage:
print(get_book_details("/works/OL14933374W"))

{
  "title": "An English forest",
  "description": null,
  "subjects": [],
  "first_sentence": null,
  "covers": [
    5688862,
    7189461
  ]
}
